In [1]:
!pipenv install scapy

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
Installing scapy...
Adding scapy to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (58b1f9) out of date, updating to (772d79)...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Locking [dev-packages] dependencies...
Updated Pipfile.lock (772d79)!
Installing dependencies from Pipfile.lock (772d79)...


In [145]:
from scapy.all import *
#load the pcap
BENCH_NO_MEDIA_RELAY="bench_without_rtpengine.pcap"
BENCH_WITH_MEDIA_RELAY="bench_with_rtpengine.pcap"
without_rtpengine_pcap = rdpcap(BENCH_NO_MEDIA_RELAY)
with_rtpengine_pcap = rdpcap(BENCH_WITH_MEDIA_RELAY)

In [146]:
import re

In [157]:
def generate_report(pkts):
    call_start_time = {}
    for pkt in pkts:
        if 'INVITE sip:1010@10.22.149.230' in str(pkt):
            call_id = re.findall(r"Call-ID: ([a-zA-Z0-9\-~]+)\\r\\n", str(pkt))[0]
            call_start_time[call_id] = pkt.time

    call_established_time = {}
    for call_id, start_time in call_start_time.items():
        for pkt in pkts:
            raw_pkt = str(raw(pkt))
            if '200 Ok' in raw_pkt and call_id in raw_pkt:
                call_established_time[call_id] = pkt.time - call_start_time[call_id]

    tmp = 0
    for time in call_established_time.values():
        tmp += time
    print(tmp / len(call_established_time))
    return call_established_time


In [158]:
without_rtpengine_report = generate_report(without_rtpengine_pcap)
with_rtpengine_report = generate_report(with_rtpengine_pcap)


2.72781065
2.6691412


In [138]:
len(without_rtpengine_report)

20